<a href="https://colab.research.google.com/github/zeynebe/AI-Summer-Camp-Project/blob/main/GlobalAIHub_DeepLearning_Urban_Sounds_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VERİ SETİ ÖNİŞLEME, 
LABELLING 
VE 
test kümelerine ayrıştırıp kaydetme



In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Working with directory library
import os
from os import path, listdir
from os.path import isdir

# We'll import the cv2(computer vision 2) and PIL (Python Imaging Library) libraries 
# to open, work and process our images
import cv2 as cv    
import PIL

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import librosa


# image data preprocessing
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# Pathlib module ensures that your file paths work the same in different operating systems
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Dropout, Flatten,Activation, BatchNormalization,MaxPooling2D
from tensorflow.keras import datasets, layers, models

# Image visualization
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
path_to_dataset = "/content/gdrive/MyDrive/spectrograms"

In [12]:
categories = ["0","1","2","3","4","5","6","7","8","9"]

In [15]:
def normalize(x):
  x = (x-x.min())/(x.max()-x.min())
  return x

In [17]:
images = []
errors_file = open("errors.txt", "a")
number = 0
error = 0


for category in categories:
  idx = categories.index(category)
  img_per_category = 0
  for image in os.listdir(f"{path_to_dataset}/{category}"):
    if img_per_category == 1000:
        break
    path_to_image = f"{path_to_dataset}/{category}/{image}"

    try:
        img = cv.imread(path_to_image, cv.IMREAD_GRAYSCALE)
        img = cv.resize(img, (128, 128))
        img = normalize(img)
        images.append([img, idx])
        
    except Exception as e:
        error += 1
        errors_file.write(f"{error} | {e}\n")
    finally:
        number += 1
    
    print(f"\rProcessed: {number} | Errors: {error}", end="")

errors_file.close()

Processed: 8732 | Errors: 0

In [18]:
len(images)

8732

In [23]:
import random

In [24]:
random.shuffle(images)

In [25]:
np.array(images)[:,1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


array([4, 1, 6, ..., 9, 0, 0], dtype=object)

In [26]:
X = []
y = []

for image, idx in images:
  X.append(image)
  y.append(idx)

In [27]:
len(X)

8732

In [28]:
len(y)

8732

In [29]:
X_train = []
y_train = []

X_val =[]
y_val = []

X_test=[]
y_test=[]

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2)
print(len(X_train),len(y_train))
print(len(X_test),len(y_test))
print(len(X_val),len(y_val))


5588 5588
1747 1747
1397 1397


In [34]:
import pickle

In [35]:
with open("X_train.pickle", "wb") as f:
  pickle.dump(X_train, f)
with open("y_train.pickle", "wb") as f:
  pickle.dump(y_train, f)

with open("X_val.pickle", "wb") as f:
  pickle.dump(X_val, f)
with open("y_val.pickle", "wb") as f:
  pickle.dump(y_val, f)

with open("X_test.pickle", "wb") as f:
  pickle.dump(X_test, f)
with open("y_test.pickle", "wb") as f:
  pickle.dump(y_test, f)

Model Hazırlanması ve Eğitimi 

In [36]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False,
    weights = "imagenet",
    input_shape = (128, 128, 3),
    pooling = "max"
)

9420800/9406464 [==============================] - 0s 0us/step


In [37]:
for layer in base_model.layers:
  layer.trainable = False

In [39]:
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

predictions = tf.keras.layers.Dense(1, activation="sigmoid")(x)

In [40]:
head_model = tf.keras.Model(inputs=base_model.inputs, outputs=predictions)

In [41]:
head_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [44]:
history = head_model.fit(
    X_train,
    y_train,
    batch_size=128,
    epochs=10,
    validation_data=(X_val, y_val)
)

ValueError: ignored

In [45]:
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_CHANNELS=3

model = Sequential([

Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)),
BatchNormalization(),
MaxPooling2D(pool_size=(2, 2)),
Dropout(0.25),

Conv2D(64, (3, 3), activation='relu'),
BatchNormalization(),
MaxPooling2D(pool_size=(2, 2)),
Dropout(0.25),

Conv2D(128, (3, 3), activation='relu'),
BatchNormalization(),
MaxPooling2D(pool_size=(2, 2)),
Dropout(0.25),

Flatten(),
Dense(512, activation='relu'),
BatchNormalization(),
Dropout(0.5),
Dense(1, activation='sigmoid'), # 2 because we have cat and dog classes
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout_6 (Dropout)         (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 61, 61, 64)       2